In [1]:
# environment: labelmaker

## Mask3D

Type of object, colour and ID in the segmentation respect the indications specified in this file: https://github.com/ScanNet/ScanNet/blob/master/BenchmarkScripts/ScanNet200/scannet200_constants.py

In [10]:
import open3d as o3d
import os
import numpy as np
import scannet200_constants

### ARKitScenes

If you want to download something from ARKitScenes (https://github.com/apple/ARKitScenes/tree/095a1190748be2c62d3f70f211352a31eac0172c?tab=readme-ov-file), I found the list of IDs of the scenes that can be used here: https://github.com/apple/ARKitScenes/blob/095a1190748be2c62d3f70f211352a31eac0172c/raw/raw_train_val_splits.csv

Additional information are here: https://github.com/apple/ARKitScenes/blob/095a1190748be2c62d3f70f211352a31eac0172c/DATA.md

First of all, one needs to download the ARKitScenes. **The following code has to be executed in the CMD, being in the root of the clone of the GitHub repository of LabelMaker**:

To set up the scene:

```bash
#
# Download scene
#
export TRAINING_OR_VALIDATION=Training
export SCENE_ID=47333462 # TODO TOSET: change with what you need

# activate environment
conda activate labelmaker

# modify the paths as you need in the following line
python 3rdparty/ARKitScenes/download_data.py raw --split $TRAINING_OR_VALIDATION --video_id $SCENE_ID --download_dir ~/Desktop/LabelMaker/ARKitScenes/ --raw_dataset_assets lowres_depth confidence lowres_wide.traj lowres_wide lowres_wide_intrinsics vga_wide vga_wide_intrinsics # TODO TOSET: in case change the paths and the things to be downloaded


#
# Convert scene to LabelMaker workspace
#
# modify the paths as you need in the following two lines
WORKSPACE_DIR=~/Desktop/LabelMaker/workspace/$SCENE_ID # TODO TOSET: change the path, in case

python scripts/arkitscenes2labelmaker.py --scan_dir ~/Desktop/LabelMaker/ARKitScenes/raw/$TRAINING_OR_VALIDATION/$SCENE_ID --target_dir $WORKSPACE_DIR # TODO TOSET: change the path, in case
```
	
	
In my case, I will find the things to work with in this directory: `/local/home/gmarsich/Desktop/LabelMaker/workspace/`

Then, to visualise the point cloud obtained after having used `scripts/arkitscenes2labelmaker.py`:

In [8]:
scene_id_num = 47333462 # TODO TOSET: change if needed
scene_id = str(scene_id_num)

# Print the original point cloud
pcd_original = o3d.io.read_point_cloud(os.path.join('/local/home/gmarsich/Desktop/LabelMaker/workspace', scene_id, 'mesh.ply')) # TODO TOSET: change the name of the point cloud to open
o3d.visualization.draw_geometries([pcd_original])

Now we want to perform the segmentation with Mask3D, that is also a part of the pipeline of LabelMaker. **The following code has to be executed in the CMD, being in the root of the clone of the GitHub repository of LabelMaker**:

```bash
python models/mask3d_inst.py --workspace $WORKSPACE_DIR # TODO TOSET: be sure that the WORKSPACE_DIR is what you need. Be aware that depending on the scene you have to change WORKSPACE_DIR
```

In my case, I will find this in the directory: `~/Desktop/LabelMaker/workspace/$SCENE_ID/intermediate`

To visualise the point cloud with segmentation:

In [9]:
#
# Get the list of objects that appear in the segmentation and their colour (use the files provided from the work of Mask3D).
#

path_predictions = os.path.join("/local/home/gmarsich/Desktop/LabelMaker/workspace", str(scene_id), "intermediate/scannet200_mask3d_1/predictions.txt")

# Read the content of the file
with open(path_predictions, 'r') as file:
    lines = file.readlines()

# Initialize the predictions array
predictions = []

# Process each line
for line in lines:
    parts = line.strip().split()  # Split the line into parts
    filename = parts[0]  # Get the filename part
    file_number = filename.split('/')[1].split('.')[0]  # Extract the number from the filename
    object_ID = int(parts[1])  # Get the number after .txt
    score = float(parts[2])  # Get the last number

    # Append the extracted information as an array to the predictions list
    predictions.append([file_number, object_ID, score])





# An alternative could be to get the colours from the point cloud and search for their assciated IDs (and name of the object) on
# https://github.com/ScanNet/ScanNet/blob/master/BenchmarkScripts/ScanNet200/scannet200_constants.py

[['000', 33, 0.9808184504508972], ['001', 41, 0.9745919108390808], ['002', 11, 0.9729697108268738], ['003', 2, 0.9682306051254272], ['004', 40, 0.9651244282722473], ['005', 40, 0.9651244282722473], ['006', 40, 0.9651244282722473], ['007', 40, 0.9651244282722473], ['008', 15, 0.9525947570800781], ['009', 13, 0.9516007304191589], ['010', 64, 0.9515623450279236], ['011', 40, 0.9465606808662415], ['012', 19, 0.9420142769813538], ['013', 96, 0.9412561058998108], ['014', 5, 0.9374870657920837], ['015', 5, 0.9330196976661682], ['016', 7, 0.9229379892349243], ['017', 7, 0.9218409061431885], ['018', 16, 0.9160168170928955], ['019', 7, 0.9009466171264648], ['020', 9, 0.8869688510894775], ['021', 21, 0.8723699450492859], ['022', 11, 0.8722449541091919], ['023', 13, 0.8578157424926758], ['024', 28, 0.857663631439209], ['025', 141, 0.8315925598144531], ['026', 40, 0.820642352104187], ['027', 13, 0.7531285881996155], ['028', 15, 0.7465308308601379], ['029', 161, 0.7401318550109863], ['030', 40, 0.65

In [5]:
# Print the segmentation with Mask3D (computed by LabelMaker)
pcd_mask3D = o3d.io.read_point_cloud(os.path.join("/local/home/gmarsich/Desktop/LabelMaker/workspace", str(scene_id), "intermediate/scannet200_mask3d_1/mesh_labelled.ply")) # TODO TOSET: change the name of the point cloud to open
o3d.visualization.draw_geometries([pcd_mask3D])

## Hypersim